# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment, Environment,ScriptRunConfig
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice
from azureml.data.dataset_factory import TabularDatasetFactory

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'capstone'

experiment=Experiment(ws, experiment_name)

In [3]:
path = "https://raw.githubusercontent.com/ewellmann/capstone/master/starter_file/heart_failure_clinical_records_dataset.csv"
ds = TabularDatasetFactory.from_delimited_files(path)
data = ds.to_pandas_dataframe().dropna()

In [4]:
# Compute name should contain only letters, digits, hyphen and should be 2-16 charachters long
compute_name = "capstonecompute"


# Check if the compute cluster already exists, if not create a new one
if compute_name not in ws.compute_targets:
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2S_V3",
                                                           min_nodes=0,
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, compute_name, compute_config)
    compute_target.wait_for_completion(show_output=True)
else:
    compute_target = ws.compute_targets[compute_name]

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [9]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling(
    {
        '--C': choice(0.01,0.1,1,10,20,50),
        '--max_iter': choice(50,100,500)
    }
)

#TODO: Create your estimator and hyperdrive config
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yaml')
estimator = ScriptRunConfig(source_directory="./.",
                      script='train.py',
                      compute_target=compute_target,
                      environment=sklearn_env)

hyperdrive_config = HyperDriveConfig(run_config = estimator,
                hyperparameter_sampling=param_sampling, 
                primary_metric_name='Accuracy',
                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                policy=early_termination_policy,
                max_total_runs=16,
                max_concurrent_runs=4)


In [10]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(hyperdrive_config)


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [11]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

KeyError: 'log_files'

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [14]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()

# Get all metrics of the best run
best_run_metrics = best_run.get_metrics()

print('Run ID: ', best_run.id)
# Print all metrics of the best run
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

Run ID:  HD_59ba39bc-f179-450a-a637-4174dbed38c9_0
Regularization Strength: 0.01
Max iterations: 50
Accuracy 0.7777777777777778


In [15]:
#TODO: Save the best model

best_model = best_run.register_model(model_path='outputs/model.pkl', model_name='capstone_model_hyperdrive', 
                                properties={'Accuracy': best_run_metrics['Accuracy'],
                                            'Regularization Strength': best_run_metrics['Regularization Strength:'],
                                           'Max Iterations': best_run_metrics['Max iterations:']})

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

